In [9]:
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import joblib

In [10]:
data = pd.read_csv("main_process_data.csv")
data

,Gender,Height,Weight,BustorChest,Cup_Size,Waist,Hips,BodyShapeIndex
0,Female,6.5,103,35,B,28,40,2
1,Male,4.6,61,34,D,21,45,2
2,Male,5.5,84,31,D,32,43,3
3,Female,6.6,93,48,E,38,34,3
4,Female,6.5,92,37,C,32,38,1
...,...,...,...,...,...,...,...,...
4995,Male,6.6,59,43,D,35,42,1
4996,Female,4.7,54,33,B,21,48,3
4997,Female,4.6,67,44,C,29,38,1
4998,Male,6.5,100,46,D,33,38,3


In [11]:
X = data.drop('BodyShapeIndex', axis=1)
y = data['BodyShapeIndex']

numeric_features = ['Height', 'Weight', 'BustorChest', 'Waist', 'Hips']
categorical_features = ['Gender', 'Cup_Size']

numeric_transformer = Pipeline(steps=[
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ('onehot', OneHotEncoder(drop='first'))
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ])


In [13]:
X_preprocessed = preprocessor.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X_preprocessed, y, test_size=0.2, random_state=42)

In [14]:
model = tf.keras.Sequential([
    tf.keras.layers.Dense(32, input_shape=(X_train.shape[1],), activation='relu'),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(1)  # Output layer for regression
])


model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])

c:\Users\833_SOURIN\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:88: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [15]:
model.fit(X_train, y_train, epochs=25, batch_size=8, validation_data=(X_test, y_test))

Epoch 1/25
500/500 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - loss: 1.3532 - mae: 0.9407 - val_loss: 0.6923 - val_mae: 0.7090
Epoch 2/25
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.7186 - mae: 0.7254 - val_loss: 0.6698 - val_mae: 0.6956
Epoch 3/25
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.6758 - mae: 0.7020 - val_loss: 0.6620 - val_mae: 0.6889
Epoch 4/25
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.6590 - mae: 0.6867 - val_loss: 0.6688 - val_mae: 0.6979
Epoch 5/25
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.6514 - mae: 0.6854 - val_loss: 0.6689 - val_mae: 0.6957
Epoch 6/25
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.6641 - mae: 0.6926 - val_loss: 0.6680 - val_mae: 0.6978
Epoch 7/25
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.6648 - mae: 0.6918 - val_loss: 0.6606 - val_mae: 0.6825
Epoch 8/25
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.6539 - mae: 0.6881 - val_loss: 0.6573 - val_mae: 0.6844
Epoch 9/25
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - lo

In [16]:
#Keras Model
model.save('body_shape_index_model.keras')
#h5 Model
# model.save('body_shape_index_model.h5')
joblib.dump(preprocessor, 'preprocessor.pkl')

['preprocessor.pkl']

In [17]:
y_pred = model.predict(X_test)
print(f'Mean Squared Error: {mean_squared_error(y_test, y_pred)}')

32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step
Mean Squared Error: 0.7138004027493343
